In [2]:
import json
import os
import sys
from dotenv import load_dotenv

sys.path.append("../")
from fns import get_secret
load_dotenv(dotenv_path="../.env")

OPENAI_PERSONAL_KEY = os.getenv("OPENAI_PERSONAL_KEY")
with open("../constants.json", encoding="utf8") as f:
    config = json.load(f)
    LLM_API_VERSION = config["openai_api_version"]
    LLM_MODEL = config["openai_api_deployment"]

OPENAI_API_KEY = get_secret("azure-openai-api-key")
OPENAI_INSTANCE = get_secret("azure-openai-instance-name")
OPENAI_SUBDOMAIN = get_secret('azure-openai-instance-name')
OPENAI_BASE_URL = f"https://{OPENAI_SUBDOMAIN}.openai.azure.com"

In [10]:
import openai
import re
import httpx
from fns import messages_prompt, get_embeddings, summarize
from tools import web_search, page_scraper, search, pick

openai.api_key = OPENAI_PERSONAL_KEY
 
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = messages_prompt(self.messages)
        return completion

prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Available Actions:

llm_eloquent:
e.g. llm_eloquent: This is a lot of text that needs to be summarized...
    features:
        - Summarizes text to make it more concise and tailored to the user's request.
        - Summarizes a page.
        - Rephrases a procured chunk of text to address their question as a helpful response.
        - When text needs to be summarized, I can pull out only the relevant information.

llm_pick: 
e.g. llm_pick: [{{ title: "Title 1", url: "http://example.com/1", content: "include the content of the result", ...rest of the result...}}, ...many more results...]
    features:
        - When a list of content chunks are provided, this action picks the item that is most relevant chunk to the user's request.
        - When a list of URLs is available, picks one that is most relevant.
        - Picks an item from a list that best suits a user's query.
        - Does not provide the full text of the result, just an identifier.

search_vtx_com: 
e.g. search_vtx_com: What products does vertex sell?
    features:
        - For general inquiries about Vertex's products or services. Returns a list of search results.
        - Questions about sales and marketing materials.
        - Not for information on a specific Vertex product.
        - Returns a list of search results of pages with only the following attributes per result: title, URL, and snippet.

search_vtx_kb:
e.g. search_vtx_kb: What are the features of Indirect Tax O Series? 
    features:
        - Search for specific/esoteric information about a tax topic.
        - Returns a list of search results, each containing completed chunks of knowledge.
        - Returns a list containing results that each have more detailed information about Vertex's products and services. 
        - Typically don't require elaboration.

elaborate_vtx_url:
e.g. elaborate_vtx_url: https://www.vertexinc.com/solutions/vertex-indirect-tax-o-series
    features:
        - Takes a single URL and returns the full text of the page.
        - Used when very detailed information is needed.
        - Returns the full content of an entire web page.
        - Get's all the context surrounding a particular topic.

calculate:
e.g. calculate: 4 * 7 / 3
    features:
        - Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
    features:
        - Returns a summary from searching Wikipedia


Example session 1:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris


Example session 2:

Question: What products does Vertex sell?
Thought: I should search for information about the products that Vertex sells.
Action: search_vtx_com: What products does Vertex sell?
PAUSE

Observation:
 [{{'title': 'Tax Technology Software & Solutions for Business | Vertex, Inc.', 'url': 'https://www.vertexinc.com/', 'content': 'full content of the result'}}, ...full list of results...]

You then output:

Thought: I should get the full text of the result that best matches the user's query.
Action: elaborate_vtx_url: https://www.vertexinc.com/
PAUSE

Observation: ...full text of the page...

You then output:

Thought: I should summarize the text to make it more concise and tailored to the user's request.
Action: llm_eloquent: What products does Vertex sell? ...full text of the page...
PAUSE

Observation: ...summary of the page...

You then output:

Answer: Vertex sells comprehensive, integrated tax technology solutions, having helped 10,000+ businesses since 1978.
""".strip()


action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions(question):
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions(question)[action](action_input)
            print("\nObservation:\n", observation, "\n")
            next_prompt = "Observation: {}".format(observation)
        else:
            return


def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def calculate(what):
    return eval(what)

def essential(results):
    return [r["content"] for r in results]

def known_actions(query): 
    return {
        "wikipedia": wikipedia,
        "calculate": calculate,
        "web_search": web_search,
        "search_vtx_com": lambda x: web_search(x + " site:vertexinc.com"),
        "search_vtx_kb": lambda x: essential(search(embeddings=get_embeddings(x)["data"][0]["embedding"])["value"]),
        "elaborate_vtx_url": lambda x: "".join(page_scraper(x).split("\n")[:100]),
        "llm_eloquent": lambda x: summarize(f"{query}\n{x}"),
        "llm_pick": lambda x: pick(query, x)
    }


<>:134: SyntaxWarning: invalid escape sequence '\w'
<>:134: SyntaxWarning: invalid escape sequence '\w'
/var/folders/lx/jtpn_5mn1t90ttxqld_yjlnw0000gq/T/ipykernel_89679/1046513109.py:134: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


In [6]:
query("Fifteen * twenty five")

Thought: The user wants me to calculate the product of fifteen and twenty-five.
Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25

Observation:
 375 

Answer: The product of fifteen and twenty-five is 375.


In [7]:
query("What products does vertex sell?")    

Thought: I should search for information about the products that Vertex sells.
Action: search_vtx_com: What products does Vertex sell?
PAUSE
 -- running search_vtx_com What products does Vertex sell?
web_search: Searching for: What products does Vertex sell? site:vertexinc.com
Found 12 results for: What products does Vertex sell? site:vertexinc.com

Observation:
 [{'title': 'Tax Technology Software & Solutions for Business | Vertex, Inc.', 'url': 'https://www.vertexinc.com/', 'snippet': '**Vertex** is the leading and most-trusted provider of comprehensive, integrated tax technology solutions, having helped 10,000+ businesses since 1978.'}, {'title': 'Learn about Vertex | Vertex Inc.', 'url': 'https://www.vertexinc.com/company/about-us', 'snippet': 'Discover **Vertex**, the leading and most-trusted provider of comprehensive, integrated tax technology solutions, having helped 10,000+ businesses since 1978.'}, {'title': 'Vertex for Marketplaces for VAT/GST Compliance | Vertex Inc', 'url':

In [8]:
query("How do I contact Vertex Inc customer service?")

Thought: The user is asking for information on how to contact Vertex Inc customer service. I should search for this information.
Action: search_vtx_com: How do I contact Vertex Inc customer service?
PAUSE
 -- running search_vtx_com How do I contact Vertex Inc customer service?
web_search: Searching for: How do I contact Vertex Inc customer service? site:vertexinc.com
Found 9 results for: How do I contact Vertex Inc customer service? site:vertexinc.com

Observation:
 [{'title': 'Contact Us for Support | Vertex Inc.', 'url': 'https://www.vertexinc.com/contact-us', 'snippet': 'For Vertex Cloud support: **Phone: 1-855-221-5885** Email: cloudsupport@vertexinc.com'}, {'title': 'Support & Services | Vertex, Inc.', 'url': 'https://www.vertexinc.com/support-services', 'snippet': '**Vertex** is the leading and most-trusted provider of comprehensive, integrated tax technology solutions, having helped 10,000+ businesses since 1978.'}, {'title': 'Tax Technology Software & Solutions for Business | V

In [11]:
query("How do you calculate tax in the state of PA?", max_turns=7)

Thought: I should search for information on how to calculate tax in the state of PA.
Action: search_vtx_kb: How to calculate tax in Pennsylvania?
PAUSE
 -- running search_vtx_kb How to calculate tax in Pennsylvania?

Observation:
 ['   Look up tax rates by jurisdiction  Searching by jurisdiction returns only the rates for the selected jurisdictions. To  look up rates for a jurisdiction:  1. Navigate to Tools > Rate Lookup > By Jurisdiction. 2. Click next to the Jurisdictions field to display the Select Jurisdictions dialog box and select jurisdictions for  which you want to see rates. 3. To refine your search, click Advanced Search. See "Advanced Search criteria" in this article for details. 4. Click Search. All tax-rate data that O Series found for the criteria displays.  This example requests the tax rates for both Pennsylvania and Philadelphia and is  useful for seeing the components of the total 8% sales tax that is charged for purchases  in Philadelphia.     Look up tax rates by T